In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from policy_evaluator_disjoint_lin_ucb import policy_evaluator_disjoint_lin_ucb

In [2]:
data = pd.read_csv('../Data/dataset_1000_minimum_reviews.csv', index_col='Unnamed: 0')
data

,user_id,movie_id,rating,genre,gender,age,occupation,time,binary_rating
0,3669,1197,4,Action|Adventure|Comedy|Romance,M,25,14,0,0
1,3621,780,2,Action|Sci-Fi|War,M,18,4,1,0
2,4917,1234,3,Comedy|Crime,M,45,0,2,0
3,1147,1544,2,Action|Adventure|Sci-Fi|Thriller,M,25,20,3,0
4,262,3175,5,Adventure|Comedy|Sci-Fi,F,25,1,4,1
...,...,...,...,...,...,...,...,...,...
284267,2240,1544,5,Action|Adventure|Sci-Fi|Thriller,M,45,12,284267,1
284268,5550,1089,4,Crime|Thriller,M,35,15,284268,0
284269,5988,1197,4,Action|Adventure|Comedy|Romance,M,25,15,284269,0
284270,3764,923,5,Drama,M,25,1,284270,1


### User features Data

In [3]:
data

,user_id,movie_id,rating,genre,gender,age,occupation,time,binary_rating
0,3669,1197,4,Action|Adventure|Comedy|Romance,M,25,14,0,0
1,3621,780,2,Action|Sci-Fi|War,M,18,4,1,0
2,4917,1234,3,Comedy|Crime,M,45,0,2,0
3,1147,1544,2,Action|Adventure|Sci-Fi|Thriller,M,25,20,3,0
4,262,3175,5,Adventure|Comedy|Sci-Fi,F,25,1,4,1
...,...,...,...,...,...,...,...,...,...
284267,2240,1544,5,Action|Adventure|Sci-Fi|Thriller,M,45,12,284267,1
284268,5550,1089,4,Crime|Thriller,M,35,15,284268,0
284269,5988,1197,4,Action|Adventure|Comedy|Romance,M,25,15,284269,0
284270,3764,923,5,Drama,M,25,1,284270,1


In [4]:
user_features = data[['user_id', 'gender', 'age', 'occupation']]

dummies_occup = pd.get_dummies(user_features['occupation'])
dummies_occup = dummies_occup.astype(int)

occupations_columns = [f'occup_{i}' for i in range(21)]
dummies_occup.columns = occupations_columns

dummies_genre = pd.get_dummies(user_features['gender'])
dummies_genre = dummies_genre.astype(int)

user_features = pd.concat([user_features.drop(['gender','occupation'], axis = 1),
                           dummies_genre, dummies_occup, ], axis=1)


In [5]:
# We first get the columns necessary to compute our vector x of the data of users
user_features = data[['time', 'movie_id', 'user_id', 'binary_rating', 'gender', 'age', 'occupation']]

# We create a dataframe where the different occupations are the columns and each row is a user
# In this dataframe, a cell returns the bool True if an user as the occupation_i and False else
dummies_occup = pd.get_dummies(user_features['occupation'])
# We replace the bools by 1s and 0s
dummies_occup = dummies_occup.astype(int)

# Same for the genders
occupations_columns = [f'occup_{i}' for i in range(21)]
dummies_occup.columns = occupations_columns

dummies_genre = pd.get_dummies(user_features['gender'])
dummies_genre = dummies_genre.astype(int)

# We drop the previous columns genders and occupation and concat our new dataframe
user_features = pd.concat([user_features.drop(['gender','occupation'], axis = 1),
                           dummies_genre, dummies_occup, ], axis=1)

# We center and reduce 'age'
user_features['age'] = (user_features['age']- user_features['age'].mean()) / user_features['age']

# Here we can indeed see that new columns were defined for the 'gender_F' and 'gender_M' as well as the 20 occupations
user_features.head() # That's the collection of our 'x' vectors for all 'time'

,time,movie_id,user_id,binary_rating,age,F,M,occup_0,occup_1,occup_2,...,occup_11,occup_12,occup_13,occup_14,occup_15,occup_16,occup_17,occup_18,occup_19,occup_20
0,0,1197,3669,0,-0.208675,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1,780,3621,0,-0.678715,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,1234,4917,0,0.328514,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1544,1147,0,-0.208675,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,4,3175,262,1,-0.208675,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### Stream of Data

In [7]:
dataframe = user_features

In [8]:
dataframe.head()

,time,movie_id,user_id,binary_rating,age,F,M,occup_0,occup_1,occup_2,...,occup_11,occup_12,occup_13,occup_14,occup_15,occup_16,occup_17,occup_18,occup_19,occup_20
0,0,1197,3669,0,-0.208675,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1,780,3621,0,-0.678715,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,1234,4917,0,0.328514,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1544,1147,0,-0.208675,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,4,3175,262,1,-0.208675,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
alpha_values = np.arange(0.05, 1.45, 0.05)

In [10]:
alpha_values

array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  , 1.05, 1.1 ,
       1.15, 1.2 , 1.25, 1.3 , 1.35, 1.4 ])

In [20]:
for alpha_value in alpha_values:
    payoffs = policy_evaluator_disjoint_lin_ucb(dataframe, alpha=alpha_value)
    pd.DataFrame(payoffs).to_csv(f'../Plots/Data/linucb/linucb_{alpha_value}.csv')